# **FUSIÓN DE DATOS HIDROLÓGICOS Y METEOROLÓGICOS**

Este notebook fusiona los datos de la Presa Abelardo L. Rodríguez con los datos meteorológicos para crear un dataset completo.

**Requisitos previos:**
- Ejecutar notebook 1.0: `datos_presa_arlso.csv`
- Ejecutar notebook 2.0: `datos_meteorologicos_completos.csv`

**Salida:** `../data/processed/datos_hidrologicos_completos.csv`

## 1. Importar librerías y configurar rutas

In [25]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

# Configurar rutas siguiendo estructura Cookiecutter
project_dir = Path.cwd().parent
data_raw = project_dir / 'data' / 'raw'
data_processed = project_dir / 'data' / 'processed'

# Crear directorio processed si no existe
data_processed.mkdir(parents=True, exist_ok=True)

print(f"Directorio de datos raw: {data_raw}")
print(f"Directorio de datos processed: {data_processed}")

Directorio de datos raw: C:\Users\Santy\Documents\GitHub\arhbpalr\arhbpalr\data\raw
Directorio de datos processed: C:\Users\Santy\Documents\GitHub\arhbpalr\arhbpalr\data\processed


## 2. Cargar datos de la presa

In [26]:
# Cargar datos de almacenamiento de la presa
file_presa = data_raw / 'datos_presa_arlso.csv'

if not file_presa.exists():
    raise FileNotFoundError(
        f"❌ No se encontró {file_presa}\n"
        "   Ejecuta primero el notebook 1.0-mcd-obtencion-datos-presa.ipynb"
    )

df_presa = pd.read_csv(file_presa)
df_presa['fecha'] = pd.to_datetime(df_presa['fecha'])

print(f"✓ Datos de presa cargados")
print(f"  Registros: {len(df_presa):,}")
print(f"  Columnas: {list(df_presa.columns)}")
print(f"  Rango: {df_presa['fecha'].min().date()} a {df_presa['fecha'].max().date()}")
print(f"\nPrimeros registros:")
display(df_presa.head())

✓ Datos de presa cargados
  Registros: 24,631
  Columnas: ['fecha', 'almacenamiento_hm3']
  Rango: 1947-04-14 a 2024-09-19

Primeros registros:


,fecha,almacenamiento_hm3
0,1947-04-14,0.09
1,1947-04-15,0.11
2,1947-04-16,0.11
3,1947-04-17,0.12
4,1947-04-18,0.12


## 3. Cargar datos meteorológicos

In [27]:
# Cargar datos meteorológicos completos
file_meteo = data_raw / 'datos_meteorologicos_completos.csv'

if not file_meteo.exists():
    raise FileNotFoundError(
        f"❌ No se encontró {file_meteo}\n"
        "   Ejecuta primero el notebook 2.0-mcd-obtencion-datos-meteorologicos.ipynb"
    )

df_meteo = pd.read_csv(file_meteo)
df_meteo['fecha'] = pd.to_datetime(df_meteo['fecha'])

print(f"✓ Datos meteorológicos cargados")
print(f"  Registros: {len(df_meteo):,}")
print(f"  Columnas: {list(df_meteo.columns)}")
print(f"  Rango: {df_meteo['fecha'].min().date()} a {df_meteo['fecha'].max().date()}")
print(f"\nPrimeros registros:")
display(df_meteo.head())

✓ Datos meteorológicos cargados
  Registros: 31,047
  Columnas: ['fecha', 'precipitacion_mm', 'lluvia_mm', 'evapotranspiracion_mm', 'temp_media_c', 'temp_max_c', 'temp_min_c', 'horas_precipitacion', 'viento_max_km_h', 'viento_medio_km_h', 'deficit_presion_vapor_kpa', 'radiacion_solar_mj_m2', 'humedad_relativa_pct', 'cobertura_nubes_pct', 'humedad_suelo_0_100cm', 'humedad_suelo_0_7cm', 'humedad_suelo_28_100cm', 'humedad_suelo_7_28cm']
  Rango: 1940-01-01 a 2024-12-31

Primeros registros:


,fecha,precipitacion_mm,lluvia_mm,evapotranspiracion_mm,temp_media_c,temp_max_c,temp_min_c,horas_precipitacion,viento_max_km_h,viento_medio_km_h,deficit_presion_vapor_kpa,radiacion_solar_mj_m2,humedad_relativa_pct,cobertura_nubes_pct,humedad_suelo_0_100cm,humedad_suelo_0_7cm,humedad_suelo_28_100cm,humedad_suelo_7_28cm
0,1940-01-01,0.0,0.0,3.19,15.5,23.7,7.9,0.0,11.9,6.8,2.33,14.54,42,41,0.053,0.053,0.051,0.062
1,1940-01-02,0.0,0.0,2.86,15.5,23.3,8.4,0.0,13.0,7.2,2.30,11.71,42,77,0.053,0.053,0.051,0.061
2,1940-01-03,0.0,0.0,3.68,17.0,25.5,9.5,0.0,18.2,9.7,2.60,14.10,42,63,0.053,0.052,0.051,0.061
3,1940-01-04,0.0,0.0,2.75,16.1,22.8,9.2,0.0,13.2,6.5,2.00,11.94,52,68,0.053,0.052,0.051,0.061
4,1940-01-05,0.1,0.1,2.19,15.1,21.0,10.0,1.0,19.8,11.3,1.38,8.83,66,78,0.053,0.052,0.051,0.061


## 4. Análisis de cobertura temporal

Antes de fusionar, veamos qué rangos de fechas tiene cada dataset.

In [28]:
print("=" * 70)
print("ANÁLISIS DE COBERTURA TEMPORAL")
print("=" * 70)

# Rangos de fechas
presa_min, presa_max = df_presa['fecha'].min(), df_presa['fecha'].max()
meteo_min, meteo_max = df_meteo['fecha'].min(), df_meteo['fecha'].max()

print(f"\n📊 Datos de presa:")
print(f"   Inicio: {presa_min.date()}")
print(f"   Fin: {presa_max.date()}")
print(f"   Días: {(presa_max - presa_min).days:,}")

print(f"\n🌤️  Datos meteorológicos:")
print(f"   Inicio: {meteo_min.date()}")
print(f"   Fin: {meteo_max.date()}")
print(f"   Días: {(meteo_max - meteo_min).days:,}")

# Calcular intersección y diferencias
interseccion_min = max(presa_min, meteo_min)
interseccion_max = min(presa_max, meteo_max)

print(f"\n🔗 Período de intersección (datos en común):")
print(f"   Inicio: {interseccion_min.date()}")
print(f"   Fin: {interseccion_max.date()}")
print(f"   Días: {(interseccion_max - interseccion_min).days:,}")

# Fechas únicas en cada dataset
fechas_presa = set(df_presa['fecha'].dt.date)
fechas_meteo = set(df_meteo['fecha'].dt.date)

solo_presa = len(fechas_presa - fechas_meteo)
solo_meteo = len(fechas_meteo - fechas_presa)
comunes = len(fechas_presa & fechas_meteo)

print(f"\n📈 Análisis de fechas únicas:")
print(f"   Fechas solo en presa: {solo_presa:,}")
print(f"   Fechas solo en meteo: {solo_meteo:,}")
print(f"   Fechas en común: {comunes:,}")

print("\n" + "=" * 70)

ANÁLISIS DE COBERTURA TEMPORAL

📊 Datos de presa:
   Inicio: 1947-04-14
   Fin: 2024-09-19
   Días: 28,283

🌤️  Datos meteorológicos:
   Inicio: 1940-01-01
   Fin: 2024-12-31
   Días: 31,046

🔗 Período de intersección (datos en común):
   Inicio: 1947-04-14
   Fin: 2024-09-19
   Días: 28,283

📈 Análisis de fechas únicas:
   Fechas solo en presa: 0
   Fechas solo en meteo: 6,416
   Fechas en común: 24,631



## 5. Estrategia de fusión

### **Estrategia seleccionada: OUTER JOIN (Unión completa)**

**Decisión:** Usaremos `merge` con `how='outer'` para preservar **todas las fechas** de ambos datasets.

**Razones:**

1. **Máxima preservación de datos**: No perdemos información de ningún dataset
2. **Análisis temporal completo**: Podemos estudiar períodos con datos meteorológicos pero sin mediciones de presa (y viceversa)
3. **Flexibilidad analítica**: Los análisis posteriores pueden decidir cómo manejar valores faltantes según el caso de uso:
   - Análisis de correlación → usar solo filas completas (`dropna()`)
   - Series temporales → interpolar valores faltantes
   - Estadísticas descriptivas → calcular por variable

**Alternativas descartadas:**
- **INNER JOIN** (`how='inner'`): Perdería ~7 años de datos de presa (1947-1940) donde no hay meteorología
- **LEFT/RIGHT JOIN**: Sesgaría el dataset hacia uno de los dos

**Manejo de valores faltantes:**
- Los valores faltantes se marcarán como `NaN`
- Se creará una columna indicadora `datos_completos` (True/False)
- Se generará un reporte de completitud por año

## 6. Fusionar datasets

In [29]:
print("Fusionando datos...\n")

# Fusión OUTER: preserva todas las fechas de ambos datasets
df_fusion = pd.merge(
    df_presa,
    df_meteo,
    on='fecha',
    how='outer',
    indicator=True  # Añade columna _merge para rastrear origen
)

# Ordenar por fecha
df_fusion = df_fusion.sort_values('fecha').reset_index(drop=True)

# Crear columna indicadora de completitud
df_fusion['datos_completos'] = (
    df_fusion['almacenamiento_hm3'].notna() & 
    df_fusion['evapotranspiracion_mm'].notna()
)

print(f"✓ Fusión completada")
print(f"  Total de registros: {len(df_fusion):,}")
print(f"  Registros con datos completos: {df_fusion['datos_completos'].sum():,}")
print(f"  Registros con datos incompletos: {(~df_fusion['datos_completos']).sum():,}")
print(f"\nOrigen de los datos:")
print(df_fusion['_merge'].value_counts())

# Eliminar columna _merge (ya no la necesitamos)
df_fusion = df_fusion.drop(columns=['_merge'])

print(f"\nPrimeros registros:")
display(df_fusion.head(10))

Fusionando datos...

✓ Fusión completada
  Total de registros: 31,047
  Registros con datos completos: 24,631
  Registros con datos incompletos: 6,416

Origen de los datos:
_merge
both          24631
right_only     6416
left_only         0
Name: count, dtype: int64

Primeros registros:


,fecha,almacenamiento_hm3,precipitacion_mm,lluvia_mm,evapotranspiracion_mm,temp_media_c,temp_max_c,temp_min_c,horas_precipitacion,viento_max_km_h,viento_medio_km_h,deficit_presion_vapor_kpa,radiacion_solar_mj_m2,humedad_relativa_pct,cobertura_nubes_pct,humedad_suelo_0_100cm,humedad_suelo_0_7cm,humedad_suelo_28_100cm,humedad_suelo_7_28cm,datos_completos
0,1940-01-01,NaN,0.0,0.0,3.19,15.5,23.7,7.9,0.0,11.9,6.8,2.33,14.54,42,41,0.053,0.053,0.051,0.062,False
1,1940-01-02,NaN,0.0,0.0,2.86,15.5,23.3,8.4,0.0,13.0,7.2,2.30,11.71,42,77,0.053,0.053,0.051,0.061,False
2,1940-01-03,NaN,0.0,0.0,3.68,17.0,25.5,9.5,0.0,18.2,9.7,2.60,14.10,42,63,0.053,0.052,0.051,0.061,False
3,1940-01-04,NaN,0.0,0.0,2.75,16.1,22.8,9.2,0.0,13.2,6.5,2.00,11.94,52,68,0.053,0.052,0.051,0.061,False
4,1940-01-05,NaN,0.1,0.1,2.19,15.1,21.0,10.0,1.0,19.8,11.3,1.38,8.83,66,78,0.053,0.052,0.051,0.061,False
5,1940-01-06,NaN,0.0,0.0,2.19,13.5,19.3,8.8,0.0,10.8,7.3,1.33,12.56,75,67,0.053,0.052,0.051,0.061,False
6,1940-01-07,NaN,0.0,0.0,2.62,13.2,19.9,5.8,0.0,8.7,5.9,1.66,14.40,59,35,0.053,0.054,0.051,0.061,False
7,1940-01-08,NaN,0.0,0.0,3.00,15.4,23.2,7.9,0.0,10.4,7.1,2.02,14.05,49,72,0.053,0.055,0.051,0.061,False
8,1940-01-09,NaN,0.0,0.0,2.99,15.6,22.4,9.4,0.0,15.1,7.2,1.76,13.85,54,50,0.053,0.054,0.051,0.061,False
9,1940-01-10,NaN,0.0,0.0,3.00,14.2,21.6,6.8,0.0,11.2,7.6,1.94,14.73,53,71,0.053,0.054,0.051,0.061,False


## 7. Análisis de valores faltantes

In [30]:
print("=" * 70)
print("ANÁLISIS DE VALORES FALTANTES")
print("=" * 70)

# Contar valores faltantes por columna
missing = df_fusion.isnull().sum()
missing_pct = (missing / len(df_fusion) * 100).round(2)

missing_df = pd.DataFrame({
    'Columna': missing.index,
    'Valores faltantes': missing.values,
    'Porcentaje': missing_pct.values
})

# Mostrar solo columnas con valores faltantes
missing_df = missing_df[missing_df['Valores faltantes'] > 0].sort_values(
    'Valores faltantes', ascending=False
)

if len(missing_df) > 0:
    print("\nColumnas con valores faltantes:\n")
    display(missing_df)
else:
    print("\n✓ No hay valores faltantes en ninguna columna")

# Análisis por año
df_fusion['año'] = df_fusion['fecha'].dt.year

completitud_anual = df_fusion.groupby('año').agg({
    'datos_completos': ['sum', 'count'],
    'almacenamiento_hm3': lambda x: x.notna().sum(),
    'evapotranspiracion_mm': lambda x: x.notna().sum()
})

completitud_anual.columns = ['Completos', 'Total', 'Con_presa', 'Con_meteo']
completitud_anual['Pct_completo'] = (
    completitud_anual['Completos'] / completitud_anual['Total'] * 100
).round(1)

print("=" * 70)
print("COMPLETITUD POR AÑO")
print("=" * 70)
print("\nAños con datos incompletos:")
display(completitud_anual[completitud_anual['Pct_completo'] < 100])

print("=" * 70)

ANÁLISIS DE VALORES FALTANTES

Columnas con valores faltantes:



,Columna,Valores faltantes,Porcentaje
1,almacenamiento_hm3,6416,20.67


COMPLETITUD POR AÑO

Años con datos incompletos:


,Completos,Total,Con_presa,Con_meteo,Pct_completo
año,,,,,
1940,0,366,0,366,0.0
1941,0,365,0,365,0.0
1942,0,365,0,365,0.0
1943,0,365,0,365,0.0
1944,0,366,0,366,0.0
1945,0,365,0,365,0.0
1946,0,365,0,365,0.0
1947,262,365,262,365,71.8
1960,0,366,0,366,0.0


## 8. Estadísticas descriptivas del dataset fusionado

In [22]:
print("=" * 70)
print("ESTADÍSTICAS DESCRIPTIVAS")
print("=" * 70)

# Seleccionar columnas numéricas (excluyendo 'año' y 'datos_completos')
cols_numericas = df_fusion.select_dtypes(include=[np.number]).columns
cols_numericas = [c for c in cols_numericas if c not in ['año', 'datos_completos']]

print("\nEstadísticas de variables principales:\n")
display(df_fusion[cols_numericas].describe())

print("\n" + "=" * 70)

ESTADÍSTICAS DESCRIPTIVAS

Estadísticas de variables principales:



,almacenamiento_hm3,precipitacion_mm,lluvia_mm,evapotranspiracion_mm,temp_media_c,temp_max_c,temp_min_c,horas_precipitacion,viento_max_km_h,viento_medio_km_h,deficit_presion_vapor_kpa,radiacion_solar_mj_m2,humedad_relativa_pct,cobertura_nubes_pct,humedad_suelo_0_100cm,humedad_suelo_0_7cm,humedad_suelo_28_100cm,humedad_suelo_7_28cm
count,24631.000000,31047.000000,31047.000000,31047.000000,31047.000000,31047.000000,31047.000000,31047.000000,31047.000000,31047.000000,31047.000000,31047.000000,31047.000000,31047.000000,31047.000000,31047.000000,31047.000000,31047.000000
mean,64.447475,0.564502,0.564502,5.727098,24.190466,31.442281,17.224972,0.824073,16.408593,9.186150,3.843011,21.838824,42.381100,28.426321,0.141138,0.097395,0.142603,0.150723
std,69.911783,2.721520,2.721520,2.059888,6.658277,6.729381,6.821807,2.471372,4.300854,2.478845,1.639479,6.061070,17.918705,29.593004,0.032037,0.059712,0.034085,0.040228
min,0.000000,0.000000,0.000000,0.370000,3.500000,8.400000,-1.600000,0.000000,5.000000,2.500000,0.120000,1.240000,5.000000,0.000000,0.052000,0.010000,0.047000,0.061000
25%,1.275000,0.000000,0.000000,4.060000,18.500000,26.400000,11.400000,0.000000,13.100000,7.500000,2.570000,16.690000,28.000000,1.000000,0.130000,0.071000,0.135000,0.128000
50%,42.890000,0.000000,0.000000,5.670000,24.600000,32.100000,16.500000,0.000000,16.200000,8.900000,3.700000,22.230000,40.000000,18.000000,0.138000,0.073000,0.141000,0.138000
75%,110.115000,0.000000,0.000000,7.375000,30.000000,36.900000,23.900000,0.000000,19.300000,10.500000,5.050000,27.000000,55.000000,49.000000,0.154000,0.099000,0.155000,0.161000
max,300.320000,87.900000,87.900000,13.290000,39.200000,47.100000,30.900000,24.000000,55.000000,30.600000,10.060000,32.430000,97.000000,100.000000,0.291000,0.423000,0.262000,0.413000


## 9. Guardar dataset fusionado

In [32]:
# Guardar en data/processed según estándares Cookiecutter
output_file = data_processed / 'datos_hidrologicos_completos.csv'

# Eliminar columna temporal 'año' antes de guardar
df_final = df_fusion.drop(columns=['año'])

df_final.to_csv(output_file, index=False)

print("=" * 70)
print("✅ DATASET GUARDADO EXITOSAMENTE")
print("=" * 70)
print(f"\nArchivo: {output_file}")
print(f"Registros: {len(df_final):,}")
print(f"Columnas: {len(df_final.columns)}")
print(f"Tamaño: {output_file.stat().st_size / 1024 / 1024:.2f} MB")
print(f"\nRango temporal: {df_final['fecha'].min().date()} a {df_final['fecha'].max().date()}")
print(f"\nColumnas incluidas:")
for i, col in enumerate(df_final.columns, 1):
    print(f"  {i:2d}. {col}")

✅ DATASET GUARDADO EXITOSAMENTE

Archivo: C:\Users\Santy\Documents\GitHub\arhbpalr\arhbpalr\data\processed\datos_hidrologicos_completos.csv
Registros: 31,047
Columnas: 20
Tamaño: 3.05 MB

Rango temporal: 1940-01-01 a 2024-12-31

Columnas incluidas:
   1. fecha
   2. almacenamiento_hm3
   3. precipitacion_mm
   4. lluvia_mm
   5. evapotranspiracion_mm
   6. temp_media_c
   7. temp_max_c
   8. temp_min_c
   9. horas_precipitacion
  10. viento_max_km_h
  11. viento_medio_km_h
  12. deficit_presion_vapor_kpa
  13. radiacion_solar_mj_m2
  14. humedad_relativa_pct
  15. cobertura_nubes_pct
  16. humedad_suelo_0_100cm
  17. humedad_suelo_0_7cm
  18. humedad_suelo_28_100cm
  19. humedad_suelo_7_28cm
  20. datos_completos
